# Prompt Engineering Fundamentals 

---
In this demo notebook, we are going to explore some of the Prompt Engineering Techniques for better model output generation.

We demonstrate how to use the boto3 Python SDK to work with *Amazon Bedrock Foundation Models*  We will be using the below models in this notebook;

* Amazon Nova Lite v1.0
* Amazon Nova Micro v1.0
* Anthropic Claude Haiku v3.5
---

In [ ]:
# install necessary libraries
import sys
import os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
# If the below show that you do not have these model accesses, please go to the Bedrock console to get access.
required_models = [
    "amazon.nova-lite-v1:0",
    "amazon.nova-micro-v1:0",
    "anthropic.claude-3-5-haiku-20241022-v1:0"
]
validate_model_access(required_models)

In [ ]:
# rich for pretty printing
from rich import print as rprint
from rich.markdown import Markdown

# boto3 for AWS SDK
import boto3


In [ ]:
# as with other AWS services Bedrock exists in Control Plane (CP) and Data Plane (Runtime) actions.
bedrock_cp = boto3.client("bedrock")
bedrock_runtime = boto3.client("bedrock-runtime")

#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use.  These are models that are available, and not necessarily the models that you have access to.

In [ ]:
foundation_models = bedrock_cp.list_foundation_models()
rprint(foundation_models)

---

## The Bedrock `Converse` API

Bedrock's [`Converse` API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) simplifies the calling of LLMs, providing a uniform interface across the different models. The `Converse` API offers;

* Standardized inputs and outputs across all models
* Better conversation handling via the standardize message format
* Access to additional functionality such as tool_usage

We will be using the below `Converse` API call in this notebook.

```
   response = bedrock_runtime.converse(
        modelId=model_id, # or inference_profile id
        inferenceConfig=inference_configs,
        system=system_prompts,
        messages=messages
    )
```

Let's build each of the above parameters below.

---
### `modelId` and Inference Profiles
This field indicates the model to be used.   E.g., for Amazon Nova Micro 1.0, it'd be `amazon.nova-micro-v1:0`

The `modelId` field can take either a model id or an inference profile id. [Inference profile](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles.html), allows for automatic routing of model invocations across multiple regions. With this capability, you can burst capacity across the different regions, seamlessly.  You can [create your own inference profiles](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-create.html) to track model usage across multiple users, projects, lines of business, etc.

We will be using Amazon managed inference profiles in this notebook.


In [ ]:
inf_profiles = bedrock_cp.list_inference_profiles()
rprint(inf_profiles)

---
### Inference Profiles
Let's store the Amazon managed inference profiles for each of our models in individual variables.  We will use these variables in the `modelId` field of the `Converse` API call.

```python
modelId = inf_profile_nova_micro_v1
```

In [ ]:
for inf_prof in inf_profiles['inferenceProfileSummaries']:
    if inf_prof['inferenceProfileName'] == 'US Nova Micro':
        rprint(inf_prof)
        inf_profile_nova_micro_v1 = inf_prof['inferenceProfileId']

    if inf_prof['inferenceProfileName'] == 'US Nova Lite':
        rprint(inf_prof)
        inf_profile_nova_lite_v1 = inf_prof['inferenceProfileId']

    if inf_prof['inferenceProfileName'] == 'US Anthropic Claude 3.5 Haiku':
        rprint(inf_prof)
        inf_profile_claude_haiku_v3_5 = inf_prof['inferenceProfileId']
        

rprint("This the lightest Amazon Nova LLM.  We will be using this model in most examples of this notebook.\n", inf_profile_nova_micro_v1, "\n")
rprint("This is the smallest model in the Anthropic family of models", inf_profile_claude_haiku_v3_5, "\n")
rprint("This is the midrange Amazon Nova LLM.  We will use it in the multi-modal use case.\n", inf_profile_nova_lite_v1, "\n")

---
### `inferenceConfig`
Inference parameters allow you to control the behavior of the model.  The `inferenceConfig` field is a dictionary that can contain the following parameters:

temperature (0.0 to 1.0, default 0.7):

- Lower values (like 0.0) make responses more deterministic although this is not guaranteed
- With a value of 0.0, the model will choose the most likely next token, higher values (like 1.0) will make the model choose other less likely tokens
- Higher values (like 1.0) make responses more randomized

maxTokens (integer, default 5k):
- Controls the maximum length of the model's response
- Model's response may get truncated if it exceeds this value

topP (0.0 to 1.0, default 0.9):
- Each token is sampled based on its probability of being the next token given the previous tokens
- The model will only consider the top P% cumulative probability mass for the next token
   
Note that not all models support all inference parameters



In [ ]:
inference_configs = {
    "temperature": 0.5,        # Controls randomness (0.0 to 1.0)
    "maxTokens": 2048,         # Maximum tokens in the response
    "topP": 0.9,               # Sample from the top 90% of the probability mass

}
rprint(inference_configs)

---
### `system` Prompts

System prompts are your (the developer's) directive to the LLM, as opposed to the user's.  It is useful for setting the pretext of the LLM task and defining the role of the LLM.  For example, you can set the system prompt to be a "helpful assistant" or "a sarcastic assistant".  The system prompt is not visible to the end user.





In [ ]:
system_prompts = [{"text": "You are a financial advisor of a large asset management firm."}]

---
#### `messages`

Messages are the conversation history between the user and the LLM.  The messages are a list of dictionaries, where each dictionary contains the following fields:
- `role`: The role of the message sender.  This can be either `user` or `assistant`
- `content`: The content of the message.  This can include text, images, files, etc.

Every component of the interaction is recorded as a list of messages.


In [ ]:
messages = [] # start with an empty convesation
user_msg1 = "What is the best way to save for my children's college tuitions?"

message_1 = {
    "role": "user",
    "content": [{"text": user_msg1}]
}

messages.append(message_1)  # later append outputs for a conversation

---

## Let's now call Bedrock!

Let's first try with Amazon Nova Micro v1

In [ ]:
response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_configs
)

In [ ]:
rprint(response)
rprint(response['output']['message']['content'][0]['text'])    

---

## Bedrock Standardizes Different Model Inputs
As previously mentioned, Bedrock Converse API allows the developer to invoke different models with a standard interface.  This means that the same API call can be used to invoke different models, even if they come from different model providers.

Let's now try the same call that we did with Amazon Nova Micro v1, but with Anthropic Claude Haiku v3.5

In [ ]:
response = bedrock_runtime.converse(
    modelId=inf_profile_claude_haiku_v3_5,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_configs
)

In [ ]:
rprint(response)
rprint(response['output']['message']['content'][0]['text'])   

## Prompt Engineering - Basic Techiques

In this section, we are going to explore some fundamental prompt engineering techniques.

---

### Zero-shot prompting
Zero Shot prompting describes the technique where we present a task to an LLM without giving it further examples. We therefore, expect it to perform the task without getting a prior “shot” at the task. Hence, “zero-shot” prompting. Modern LLMs demonstrate remarkable zero-shot performance and a positive correlation can be drawn between model size and zero-shot performance.

In [ ]:

# If you'd like to try your own prompt, edit this parameter!
zero_shot_system_prompt = """You are a customer service agent tasked with classifying emails by type. Please output your answer and then justify your classification. How would you categorize this email? """

zero_shot_prompt = """
<email>
I would like to know how I can contribute to my retirement account? Any additional resources to read about IRA would be helpful.
</email> 
Provide and explanation for your choice of answer

The categories are: 
(A) IRA 
(B) 529 Plan
(C) Cash Management
(D) Youth Account
"""



The xml tags above were historically used in earlier Anthropic Claude models to breakdown the prompt into logical segments. Models from other providers, and the current Anthropic Claude models 3+, do not require these xml tags. But they can still be useful in prompt construction as they allow you to reference specific parts of the prompt.  For example, you can use the `<input>` tag to reference the input text in the prompt.  This is useful for creating dynamic prompts that can change based on the input text.

In [ ]:
# Let's try this with Amazon Nova Lite v1

zero_shot_msg = {
    "role": "user",
    "content": [{"text": zero_shot_prompt}]
}

zero_shot_messages = [zero_shot_msg]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=zero_shot_messages,
    system=[{"text": zero_shot_system_prompt}],
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

In [ ]:
# Let's try the same prompt with Anthropic Claude Haiku 3.5.  Again, there is no change to the converse API call.  Only the modelId value changes.

response = bedrock_runtime.converse(
    modelId=inf_profile_claude_haiku_v3_5, # note the modelId change
    messages=zero_shot_messages,
    system=[{"text": zero_shot_system_prompt}],
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

---

### Few-shot prompting
Giving the model more information about the tasks at hand via examples is called Few-Shot Prompting. It can be used for in-context learning by providing examples of the task and the desired output. We can therefore condition the model on the examples to follow the task guidance more closely.

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
few_shot_prompt = """Your job is to generate financial summaries similar to the examples below: 

Microsoft: Microsoft reported record revenue of $51.9 billion in its latest quarterly earnings report, a 20% increase year-over-year. Its net income rose to $16.7 billion, up 21% compared to the previous year. Microsoft's revenue growth was driven by its cloud computing business Azure and Office 365 productivity tools.

Amazon: In its most recent quarterly results, Amazon reported net sales of $96.1 billion, an increase of 15% compared to the same period last year. Amazon's net income decreased to $2.9 billion, down 58% year-over-year, as the company faced rising costs due to inflation and supply chain issues. Amazon Web Services, its cloud computing segment, continued to see strong growth with sales up 33%.

Now provide a brief financial summary for Apple Inc. based on the following key facts:
- Revenue for the last 12 months: $260 billion 
- Net income: $55 billion
- Total assets: $321 billion
- Market capitalization: $2.2 trillion

"""

In [ ]:
# We will be using Amazon Nova Micro v1, unless stated otherwise.  But please feel free to try with Haiku or the other models.  
# If you wish to try models other than Amazon Nova Micro v1, Amazon Nova Lite v1 or Anthropic Claude Haiku 3.5, please ensure that you enable them first in the Bedrock console.


few_shot_message = {
    "role": "user",
    "content": [{"text": few_shot_prompt}]
}

few_shot_messages = [few_shot_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=few_shot_messages,
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

---

### Chain of Thought prompting
Chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding. The main idea of CoT is that by showing the LLM some few shot exemplars where the reasoning process is explained in the exemplars, the LLM will also show the reasoning process when answering the prompt. This explanation of reasoning often leads to more accurate results.


In [ ]:
# If you'd like to try your own prompt, edit this parameter!
cot_prompt = """
Apple Inc. key facts:
- Revenue for the last 12 months: $260 billion 
- Operating Expenses: $205 billion
- Total assets: $321 billion
- Market capitalization: $2.2 trillion

If Apple's revenue is expected to grow by 10% in the next year, however it's expenses are expected to increase by 15%, what would be the expected net income for Apple in the next year?

Think step-by-step and provide your thought process in the <thinking></thinking> XML tags and the final answer in the <answer></answer> XML tags.

"""

In [ ]:

cot_message = {
    "role": "user",
    "content": [{"text": cot_prompt}]
}

cot_messages = [cot_message] 

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=cot_messages,
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

## Prompt Engineering - Different task examples

---

### Conversation Summarization
Prompt engineering for conversation summarization is a process of crafting prompts that guide a language model like Nova in summarizing dialogues or conversations effectively.

In [ ]:
summarization_prompt = """
Provide a concise bullet-point summary of the following conversation
<conversation>
Susan: Welcome back, Chuck. This is the second part of our interview, and I want to ask you these questions as quickly as possible because I know you have a flight to catch, so. . . .

Chuck: No problem. I’m happy to chat with you.

Susan: It says on your website that you never studied business before accepting a marketing role at a Spanish-speaking multimedia publisher. Was … was that—

Chuck: Yeah.

Susan: So was that more of a stretch, or … ?

Chuck: Yeah, well, actua—

Susan: Oh, I’m sorry, just a minute. Can everyone else just put themselves on mute, please?

Akim: [inaudible 00:37]

Susan: Great, thanks. OK, so where were we? So, I was asking you if, when you took the new role as Director, it felt like a stretch?

Chuck: Right, yeah. . . . I knew there would be a period of adjustment that I just would have to push through, you know?

Susan: Absolutely. Were there any other challenges or roadblocks that you weren’t expecting though?

Chuck: Hm, roadblocks I wasn’t expecting. Um…

Susan: [laughs] I keep throwing you curveballs.

Chuck: No, it’s OK. So, I actually didn’t anticipate the workload involved in learning a foreign language on the job. It’s like, you know, sometimes. . . . I mean, you need to be really flexible and ready to change it up if your strategy isn’t actually working. Know what I mean?

Susan: [laughs] Definitely. So what was your budget like, I mean, was that, like, a challenge, too?

Chuck: Hoo boy. [laughs] Yeah it definitely was. We had funding available, but we just, um, needed so many hands on deck. It was hard to, you know, actually manage so many people with our existing resources. [coughs]

Susan: I totally understand. So what was your next move? I mean, did you—

Chuck: Yeah, yeah. . . . So next, I kind of wanted to see where I do a gap analysis and throw resources at new initiatives where we weren’t, uh, I mean, I wanted to actually close the loop in those places where we didn’t quite have a foothold.

Susan: Nice, nice.

Chuck: Yeah, we really just needed to keep what was working and then lay out a new plan for [clears throat] future growth.

Susan: So, it looks like we have time for maybe, um, let’s see— maybe one more question.

Chuck: Sounds good. Shoot.

Susan: How did this experience change the way you approached new challenges going forward?

Chuck: Hm, good one. [laughs] I think that I was less, uh, likely to question my own judgement when it came to adaptability. Now I know that I really have it in me, you know?

Really all it takes is a great marketing team underneath you and the freelance resources to put together the best marketing initiative you can afford, you know, with, um, the standard tools high-quality content, good ad placement, good SEO, a strong social presence and then maybe debut a new idea every six months. Um, like, you could come up with a certain theme or catch phrase and actually weave it through all your branding and regular initiatives.

Susan: OK, well thanks, Chuck. This is really good stuff. I know you have to run, um, but I’d just like to thank you for, you know, taking this time with me today.

Chuck: Sure, sure. Of course
</conversation>

"""

In [ ]:
summarization_message = {
    "role": "user",
    "content": [{"text": summarization_prompt}]
}

summarization_messages = [summarization_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=summarization_messages,
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

### Question & Answering
Prompt engineering for question answering is a process of crafting prompts that guide a language model like Nova in answering questions on provided context effectively. This is used as a basis for more advanced techniques such as Retrieval-Augmented Generation (RAG).

In [ ]:
qa_system_prompt = """You are a customer service agent tasked with answering questions about various financial products and services. Please output your answer and then justify your classification. """

qa_prompt = """
<context>
When the teen turns 18, their account is eligible to transition to the regular Fidelity brokerage account with expanded features like option and margin trading.
The teen will be prompted to transition their account starting on their 18th birthday. They will have 60 days to do so before their debit card and ability to trade will be restricted. Once the account is transitioned, the debit card that the teen was issued for their Fidelity Youth™ Account will continue to be valid until it expires; at that point, a new debit card will be issued.
Once a teen turns 18, the teen may choose whether or not the parent/guardian will continue to have access to that teen’s Fidelity Youth Account information. Teens can still use the Fidelity Youth™ app when they turn 18, however additional capabilities are available to them in the Fidelity mobile app.
</context> 

<question> What is the role of a parent/ guardian according to Fidelity youth accounts? </question>

"""


In [ ]:
qa_message = {
    "role": "user",
    "content": [{"text": qa_prompt}]
}

qa_messages = [qa_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=messages,
    system=[{"text": qa_system_prompt}],
    inferenceConfig=inference_configs
)

rprint(response['output']['message']['content'][0]['text'])

### Content Generation
We can use prompt engineering to guide the model in generating content that is relevant to the task at hand. This can be used for generating content for a variety of tasks such as content creation, content summarization, and more.
In this example, the model will generate a table using provided data and as well as some questions that it can answer based on this data. This approach is common for getting the model to address more complex research type tasks where multiple steps are involved in generating the final output.

In [ ]:
content_generation_prompt = """Below is research that you performed previously. On basis of this research please answer the question below. 
If it does not include the exact answer, please respond with relevant information from the research in the response. Prefer more recent sources when possible.
If the context contains publication ids, please include citations in the response. Citations should be formatted as [publication_id] or [org_id]
If the user asks for a chart, return only machine readable json structure for the chart in HighCharts format without any formatting. 
Prefer answering in First Person (I, me, my) style.
In your response, after you answer draw a line, and include 3 additional questions you can answer with the research, one question per line.
Example:
{Your Answer}

Additional questions I can answer::
Question 1
Question 2
Question 3

<research>
JPMorgan Chase Bank, N.A. (164500) Financials: 
 
|Indicator|2020(annual)|2021(annual)|2022(annual)|
|---|---|---|---|
|Total assets (USD Billion)|3025.28|3306.98|3201.94|
|Tangible Common Equity (USD Billion)|221.41|262.33|280.96|
|Problem Loans / Gross Loans|1.64|1.14|1.00|
|Tangible Common Equity / Risk Weighted Assets|16.48|18.83|19.04|
|Problem Loans / (Tangible Common Equity + Loan Loss Reserve)|6.71|4.44|3.80|
|Net Interest Margin|2.13|1.74|2.22|
|PPI / Average RWA|3.30|2.90|3.45|
|Net Income / Tangible Assets|0.70|1.16|1.09|
|Cost / Income Ratio|58.76|61.90|57.71|
|Market Funds / Tangible Banking Assets|15.38|12.38|12.43|
|Liquid Banking Assets / Tangible Banking Assets|55.38|55.31|49.91|
|Gross Loans / Due to Customers|47.24|44.01|48.72|
 
Bank of America Corporation (541000) Financials: 
 
|Indicator|2020(annual)|2021(annual)|2022(annual)|
|---|---|---|---|
|Total assets (USD Billion)|2786.01|3146.17|3024.42|
|Tangible Common Equity (USD Billion)|174.68|175.58|190.40|
|Problem Loans / Gross Loans|1.18|0.95|0.91|
|Tangible Common Equity / Risk Weighted Assets|12.74|12.55|13.49|
|Problem Loans / (Tangible Common Equity + Loan Loss Reserve)|5.92|5.26|4.87|
|Net Interest Margin|1.75|1.52|1.82|
|PPI / Average RWA|1.94|1.96|2.23|
|Net Income / Tangible Assets|0.67|0.92|0.71|
|Cost / Income Ratio|67.23|69.57|66.12|
|Market Funds / Tangible Banking Assets|21.09|21.26|22.71|
|Liquid Banking Assets / Tangible Banking Assets|57.50|58.98|55.10|
|Gross Loans / Due to Customers|54.20|50.35|56.05|
 
Wells Fargo Bank, N.A. (811500) Financials: 
 
|Indicator|2020(annual)|2021(annual)|2022(annual)|
|---|---|---|---|
|Total assets (USD Billion)|1767.81|1779.50|1717.53|
|Tangible Common Equity (USD Billion)|145.47|148.21|149.95|
|Problem Loans / Gross Loans|1.69|1.36|1.14|
|Tangible Common Equity / Risk Weighted Assets|14.36|15.35|15.34|
|Problem Loans / (Tangible Common Equity + Loan Loss Reserve)|9.18|7.37|6.45|
|Net Interest Margin|2.41|2.13|2.77|
|PPI / Average RWA|1.46|1.91|2.22|
|Net Income / Tangible Assets|0.20|1.00|0.95|
|Cost / Income Ratio|75.75|69.83|67.73|
|Market Funds / Tangible Banking Assets|5.58|3.44|6.68|
|Liquid Banking Assets / Tangible Banking Assets|41.44|41.29|35.80|
|Gross Loans / Due to Customers|61.22|57.78|65.53|
 
</research>
 
<analysis>
 
(Doc Id: PBC_1362054, Publish Date: 2023-05-11, Title:Bank of America Corporation: Update to credit analysis following ratings upgrade)
 
1. Support and Structural Considerations:
About Moody's Bank ScorecardOur Bank Scorecard is designed to capture, express and explain in summary form our Rating Committee's judgment. When read in conjunction with our research, a fulsome presentation of our judgment is expressed. As a result, the output of our Scorecard may materially differ from that suggested by raw data alone (though it has been calibrated to avoid the frequent need for strong divergence). The Scorecard output and the individual scores are discussed in rating committees and may be adjusted up or down to reflect conditions specific to each rated entity. As per Moody's Banks rating methodology, the historic ratios in the scorecard for Capital are as of most recent period, for Asset Risk and Profitability they are the worse of the most recent year-to-date period or the average of the last three years and the most recent year-to-date, and for Funding Structure and Liquid Resources they are as of the most recent year-end.
2. RATINGS:
Please see the ratings section at the end of this report for more information. The ratings and outlook shown reflect information as of the publication date. 
Summary:
Bank of America Corporation (BAC, A1 stable) is the parent holding company for the second largest banking group in the US. The group is a global systemically important bank operating primarily through its principal bank subsidiary, Bank of America N.A. (BANA, Aa1 stable, a2 baseline credit assessment). BAC's credit profile is supported by its conservative risk appetite, a balanced and diversified business mix, strong funding and liquidity, robust cost discipline, strengthened capital and resilient profitability. During 2022, in response to increases in its regulatory capital requirements, BAC boosted its capital ratios significantly.
 
</analysis>
 
<questions>
1. Can you compare the financials for JPMC, Bank of America and Wells fargo in a table format?
2. For statements referenced from documents include a reference in the format [docId].
DO NOT UNDER ANY CIRCUMSTANCES USE ANY DATA OTHER THAN MENTIONED IN THE RESEARCH SECTION.
</questions>

"""

In [ ]:
content_generation_message = {
    "role": "user",
    "content": [{"text": content_generation_prompt}]
}
content_generation_messages = [content_generation_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=content_generation_messages,
    inferenceConfig=inference_configs
)

rprint(Markdown(response['output']['message']['content'][0]['text']))

### Multi-modal prompts
In addition to text, some models allow for text and vision prompt capability.

This is useful in a number of use cases including:
- Extracting information from charts, graphs, diagrams, etc
- Summarizing rich content
- Answering questions based on images
- Being able to effective handle more complex format types. For example we can ask the model to narrate a PowerPoint presentation and then utilize the narrative in downstream semantic search applications

We will now be using Amazon Nova Lite, which allows for text and vision inputs.  Amazon Nova Micro is for text only.

Let's look at an example using a page out of [JPMC's 2022 Annual Report](data/jpmc_annual_report_page_6.pdf)

In [ ]:
# Let's take a look at the pdf document
# You can also find this document directly under "fsi-genai-bootcamp/02_prompt_engineering/data/"
# The code in this cell is just used to view the document, it is not used in the Bedrock prompt

pdf_path = "data/jpmc_annual_report_page_6.pdf"
from base64 import b64encode
from IPython.display import display, HTML

def display_pdf(pdf_path, width=1000, height=700):
    with open(pdf_path, 'rb') as f:
        pdf_bytes = f.read()
    base64_pdf = b64encode(pdf_bytes).decode('utf-8')
    display(HTML(f'<embed src="data:application/pdf;base64,{base64_pdf}" width="{width}" height="{height}" type="application/pdf"/>'))


display_pdf(pdf_path)

In [ ]:
prompt_data_text = """
Extract the data contained in the chart and provide it in a json format. The data should include the Net Income, Diluted EPS, and ROTCE from 2004 to 2022 and should be structured as follows:
{"net_income":{"units:"$B", "data_points":{"2004": 4.5}}, "diluted_eps":{...}, "rotce":{...}}

"""

In [ ]:
def read_pdf_as_bytes(pdf_path):
    """Reads a PDF file and returns its content as bytes."""
    with open(pdf_path, "rb") as pdf_file:
        pdf_bytes = pdf_file.read()
    return pdf_bytes


pdf_path = "data/jpmc_annual_report_page_6.pdf"

pdf_content = read_pdf_as_bytes(pdf_path)

In [ ]:
# note how the message has a text and an document component
# Amazon Nova multi-modal models can read text, documents and images
# Some multi-modal models have image reading capability only, and documents such as pdf will need to be converted into images beforehand
# E.g., the below message input will not work with Anthropic Claude 3.5, unless the pdf is first converted into image format

pdf_message = {
    "role": "user",
    "content": [
        {"text": prompt_data_text},
        {
            "document": {
                "format": "pdf",
                "name": "jpmc_annual_rpt",
                "source": {"bytes": pdf_content},
            }
        },
    ],
}

pdf_messages = [pdf_message]

response = bedrock_runtime.converse(
    modelId=inf_profile_nova_lite_v1,
    messages=pdf_messages,
    inferenceConfig=inference_configs,
)

rprint(response)
rprint(Markdown(response['output']['message']['content'][0]['text']))

## Conclusion

We are just getting warmed up, we explored basic prompting techiques. Please proceed to the next Notebook.